In [1]:
import pandas as pd
import os 
import pickle
import glob

In [2]:
CLUEWEB_PARTS_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed"

In [3]:
with open("tests/execution_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20231220.pkl", "rb") as input_file:
    t1 = pickle.load(input_file)

In [3]:
t1.keys()

dict_keys(['args', 'evaluation_cache'])

In [4]:
t1['evaluation_cache'].keys()

dict_keys(['Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-59-01797_4', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-82-10922_10', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-25-03447_1', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-27-16948_2', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-93-17278_14', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-96-08875_4', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-05-08217_1', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-08-10626_3', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-14-16263_0', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-14-16263_2', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-23-12229_1', 'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0001-30-20122_4',

In [5]:
t1['evaluation_cache']['Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4']

{'score': 3,
 'reasoning': 'A passagem responde diretamente à pergunta, indicando que a Praça XV de Novembro está localizada bem no centro da cidade de Florianópolis, sem incluir informações não relacionadas.',
 'usage': {'prompt_tokens': 792, 'completion_tokens': 53, 'total_tokens': 845},
 'cost': 0.02694,
 'duration': 5.071035623550415,
 'saved_cost': 0.0}

In [15]:
existing_passages = []

for key, value in t1['evaluation_cache'].items():
    
    passage_id = key.split("_")[-2] + "_" + key.split("_")[-1]
    
    existing_passages.append({"passage_id": passage_id})

In [18]:
existing_passages_df = pd.DataFrame(existing_passages)

In [21]:
id_conversion_df = pd.read_csv(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_to_clueweb_id.tsv"), sep="\t", names=['colbertx_id', 'clueweb_id'])

In [22]:
existing_passages_df = existing_passages_df.merge(id_conversion_df, left_on='passage_id', right_on='clueweb_id')[list(existing_passages_df.columns) + ['colbertx_id']]

In [25]:
collection_parts = glob.glob(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_0*.tsv"))

In [26]:
collection_parts

['/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_01.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_00.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_02.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_03.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_04.tsv']

In [27]:
all_parts = []

In [28]:
for i, collection_part in enumerate(collection_parts):
    print("Merging {}...".format(os.path.basename(collection_part)))

    part_df = pd.read_csv(collection_part, sep="\t", header=None, names=['colbertx_id', 'passage'])

    all_parts.append(existing_passages_df.merge(part_df))

Merging clueweb22-pt_colbertx_01.tsv...
Merging clueweb22-pt_colbertx_00.tsv...
Merging clueweb22-pt_colbertx_02.tsv...
Merging clueweb22-pt_colbertx_03.tsv...
Merging clueweb22-pt_colbertx_04.tsv...


In [29]:
all_parts_df = pd.concat(all_parts)

In [5]:
all_parts_df

,passage_id,colbertx_id,passage
0,clueweb22-pt0000-44-08794_4,2107121,No local onde Floripa foi inicialmente chamada...
1,clueweb22-pt0000-59-01797_4,2789724,"Ao caminhar pelas charmosas ruas do centro, vo..."
2,clueweb22-pt0000-82-10922_10,3912245,"A Ponte Hercílio Luz, cartão-postal de Florian..."
3,clueweb22-pt0000-54-01849_0,2559888,(UFPR 2011) O território brasileiro possui div...
4,clueweb22-pt0000-77-06884_1,3668516,O tamanho do Brasil e a grande variedade de cl...
...,...,...,...
2187,clueweb22-pt0002-03-04740_8,9679634,Um especialista é quem deve indicar a necessid...
2188,clueweb22-pt0002-06-04510_1,9805147,apsiganocj e mais 63 usuários acharam esta res...
2189,clueweb22-pt0001-85-13967_0,8847248,Dez coisas que podem estar deixando seu Wi-Fi ...
2190,clueweb22-pt0001-98-01939_4,9442477,A internet banda larga é a forma mais comum de...


In [31]:
all_parts_df.to_csv("tests/execution_log_20231220_passages.tsv", sep='\t', index=False)

In [4]:
all_parts_df = pd.read_csv("tests/execution_log_20231220_passages.tsv", sep='\t')

In [15]:
reverse_mapping = {}
already_analized = 0

for key, value in t1['evaluation_cache'].items():
    
    key_split = key.split("_")
    
    query = key_split[0]
    
    passage_id = key_split[-2] + "_" + key_split[-1]
    
    print("key={}\nquery={}\npassage_id={}".format(key, query, passage_id))
      
    reverse_mapping_key = "{}__{}".format(query, all_parts_df[all_parts_df['passage_id'] == passage_id]['passage'].to_numpy()[0])
    
    if reverse_mapping_key in reverse_mapping:
        print(">> Already in reverse mapping: {}".format(reverse_mapping[reverse_mapping_key]))
        print(">>> reverse_mapping_key= {}\n".format(reverse_mapping_key))
        
        reverse_mapping[reverse_mapping_key].append(key)
        
        already_analized += 1
    else:
        reverse_mapping[reverse_mapping_key] = [key]
        
    print("\n")

key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0000-44-08794_4


key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-59-01797_4
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0000-59-01797_4


key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-82-10922_10
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0000-82-10922_10


key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-25-03447_1
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0000-25-03447_1


key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-27-16948_2
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0000-27-16948_2


key=Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-93-17278_14
query=Onde está localizada a Praça XV de Novembro?
passage_id=clueweb22-pt0



key=Cuidados ao usar materiais perfurocortantes_clueweb22-pt0001-87-18523_1
query=Cuidados ao usar materiais perfurocortantes
passage_id=clueweb22-pt0001-87-18523_1


key=Cuidados ao usar materiais perfurocortantes_clueweb22-pt0001-88-12563_3
query=Cuidados ao usar materiais perfurocortantes
passage_id=clueweb22-pt0001-88-12563_3


key=Cuidados ao usar materiais perfurocortantes_clueweb22-pt0001-88-12563_6
query=Cuidados ao usar materiais perfurocortantes
passage_id=clueweb22-pt0001-88-12563_6


key=Cuidados ao usar materiais perfurocortantes_clueweb22-pt0002-02-19619_3
query=Cuidados ao usar materiais perfurocortantes
passage_id=clueweb22-pt0002-02-19619_3


key=Cuidados ao usar materiais perfurocortantes_clueweb22-pt0002-06-01668_13
query=Cuidados ao usar materiais perfurocortantes
passage_id=clueweb22-pt0002-06-01668_13


key=Delirio versus alucinação_clueweb22-pt0000-61-09104_11
query=Delirio versus alucinação
passage_id=clueweb22-pt0000-61-09104_11


key=Delirio versus alucinaçã



key=Como transformar uma cidade pacata em um polo turístico?_clueweb22-pt0001-29-12088_8
query=Como transformar uma cidade pacata em um polo turístico?
passage_id=clueweb22-pt0001-29-12088_8


key=Como transformar uma cidade pacata em um polo turístico?_clueweb22-pt0001-01-18816_8
query=Como transformar uma cidade pacata em um polo turístico?
passage_id=clueweb22-pt0001-01-18816_8


key=Como transformar uma cidade pacata em um polo turístico?_clueweb22-pt0001-90-06919_361
query=Como transformar uma cidade pacata em um polo turístico?
passage_id=clueweb22-pt0001-90-06919_361


key=Como transformar uma cidade pacata em um polo turístico?_clueweb22-pt0000-81-02764_0
query=Como transformar uma cidade pacata em um polo turístico?
passage_id=clueweb22-pt0000-81-02764_0


key=Como transformar uma cidade pacata em um polo turístico?_clueweb22-pt0001-90-06919_364
query=Como transformar uma cidade pacata em um polo turístico?
passage_id=clueweb22-pt0001-90-06919_364


key=Cuidados ao usar mate



key=Como o Brasil reagiu a epidemia de AIDS no fim do século XX?_clueweb22-pt0001-67-08445_1
query=Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
passage_id=clueweb22-pt0001-67-08445_1


key=Qual a maior torcida de futebol do Brasil?_clueweb22-pt0001-59-13213_7
query=Qual a maior torcida de futebol do Brasil?
passage_id=clueweb22-pt0001-59-13213_7


key=Quando se realizou o plebiscito popular para definir o sistema político do Brasil?_clueweb22-pt0001-90-13045_6
query=Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
passage_id=clueweb22-pt0001-90-13045_6


key=Quando se realizou o plebiscito popular para definir o sistema político do Brasil?_clueweb22-pt0001-46-16078_12
query=Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
passage_id=clueweb22-pt0001-46-16078_12


key=Quando se realizou o plebiscito popular para definir o sistema político do Brasil?_clueweb22-pt0000-28-17576_1
query=Quando se realizou 



key=Como deixar a internet rápida?_clueweb22-pt0000-70-15922_11
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-70-15922_11


key=Como deixar a internet rápida?_clueweb22-pt0000-64-15090_3
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-64-15090_3


key=Como deixar a internet rápida?_clueweb22-pt0001-18-11431_0
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0001-18-11431_0


key=Como deixar a internet rápida?_clueweb22-pt0000-34-11471_1
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-34-11471_1


key=Como deixar a internet rápida?_clueweb22-pt0000-29-01828_5
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-29-01828_5


key=Como deixar a internet rápida?_clueweb22-pt0000-82-18271_4
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-82-18271_4


key=Como deixar a internet rápida?_clueweb22-pt0000-84-17279_1
query=Como deixar a internet rápida?
passage_id=clueweb22-pt0000-84-17279_1


key=Como 

In [16]:
already_analized

28

In [10]:
t1['evaluation_cache']['Quais os sinonimos de eficiência relacionado a competências?_clueweb22-pt0000-34-13245_1']

{'score': 0,
 'reasoning': 'A passagem não fornece sinônimos de eficiência, nem está diretamente relacionada à pergunta sobre sinônimos de eficiência relacionados a competências. A passagem menciona os conceitos de eficiência e eficácia, mas não os define ou fornece sinônimos.',
 'usage': {'prompt_tokens': 584, 'completion_tokens': 78, 'total_tokens': 662},
 'cost': 0.022199999999999998,
 'duration': 8.852972030639648,
 'saved_cost': 0.0}

In [11]:
t1['evaluation_cache']['Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-68-06675_3']

{'score': 0,
 'reasoning': 'A passagem não fornece nenhuma informação sobre a localização da Praça XV de Novembro, nem mesmo está relacionada ao tema da pergunta.',
 'usage': {'prompt_tokens': 522, 'completion_tokens': 45, 'total_tokens': 567},
 'cost': 0.01836,
 'duration': 3.8660213947296143,
 'saved_cost': 0.0}

In [12]:
t1['evaluation_cache']['O que são os celulares "mid-range"?_clueweb22-pt0001-03-04486_1']

{'score': 0,
 'reasoning': "A passagem não está relacionada com a pergunta, pois trata de um aviso sobre a qualidade da tradução automática de um artigo e não fornece nenhuma informação sobre celulares 'mid-range'.",
 'usage': {'prompt_tokens': 620, 'completion_tokens': 58, 'total_tokens': 678},
 'cost': 0.02208,
 'duration': 3.0876052379608154,
 'saved_cost': 0.0}

In [13]:
t1['evaluation_cache']['Delirio versus alucinação_clueweb22-pt0000-73-02673_2']

{'score': 0,
 'reasoning': 'A passagem não tem relação alguma com a pergunta, pois trata-se de um anúncio para um serviço de respostas verificadas por especialistas, enquanto a pergunta se refere a conceitos psiquiátricos ou psicológicos.',
 'usage': {'prompt_tokens': 533, 'completion_tokens': 67, 'total_tokens': 600},
 'cost': 0.02001,
 'duration': 4.865850210189819,
 'saved_cost': 0.0}

In [14]:
len(reverse_mapping)

2164

In [17]:
t1.keys()

dict_keys(['args', 'evaluation_cache'])

In [18]:
t1['reverse_mapping'] = reverse_mapping

In [19]:
t1.keys()

dict_keys(['args', 'evaluation_cache', 'reverse_mapping'])

In [20]:
with open("tests/execution_with_reverse_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20231220.pkl", "wb") as output_file:
    pickle.dump(t1, output_file, pickle.HIGHEST_PROTOCOL)